In [ ]:
import ee
import geemap

In [3]:
## import ipywidget
import ipywidgets as widgets
from ipyleaflet import WidgetControl
from ipywidgets import interact, interactive, fixed, interact_manual

In [ ]:
#import geemap and create an interactive map 
import geemap
Map = geemap.Map(center=[35.4387,139.7750], zoom=8)
Map = geemap.Map()
# Map
# Map.split_map(left_layer='HYBRID', right_layer='ROADMAP')
# Map
# Map.add_basemap('HYBRID')  # Add Google Map
# Map

In [7]:
#create ipywidgets 
# start date 
start_date = widgets.Text(
    value='',
    placeholder='YYYY-MM-DD',
    description='Date1:',
    disabled=False
)

end_date = widgets.Text(
    value='',
    placeholder='YYYY-MM-DD',
    description='Date2:',
    disabled=False
)
# text widget for entering a particular location 
location1 = widgets.Text(
    value ='',
    placeholder = 'long,lat',
    description = 'Location:',
    disabled = False
)
# drop down widget to specify what index analysis we want to perform 
index1 = widgets.Dropdown(
    options=['NDVI', 'NDBI', 'NDWI'], value=None, description='Index:'
)

# index1.observe(dropdown_change)

In [8]:
def on_button_clicked(change):
      # Confirmation of projection
  satellite = 'COPERNICUS/S2'
  loc = location1.value   # retrieve the location values input by the user 
  loc_list = loc.split(",")
  longitude = loc_list[0]
  longitude = float(longitude)
  latitude = loc_list[1]
  latitude = float(latitude)
  pt = ee.Geometry.Point([longitude, latitude])
  areaM2 = 1e6
  region = pt.buffer(ee.Number(areaM2).sqrt().divide(2), 1).bounds() #creating region of 1km square around the location point 
  # b1 = ee.Geometry.Point([longitude, latitude])
  # b1 = ee.Geometry.Point([longitude,latitude])
  #Specify the dataset and filtering parameters
  from_date= ee.Date(start_date.value)
  to_date  =ee.Date(end_date.value)
  dataset1= ee.ImageCollection(satellite).filterDate(from_date, from_date.advance(1, 'day')).mean().clip(region) #filter(ee.Filter.geometry(bl)).filter(ee.Filter.equals('MGRS_TILE', '54SUD').clip(Jpn_region)
  dataset2 = ee.ImageCollection(satellite).filterDate(to_date, to_date.advance(1, 'day')).mean().clip(region)   # filter(ee.Filter.geometry(bl)).filter(ee.Filter.equals('MGRS_TILE', '54SUD').clip(Jpn_region)
  vis = {"bands": ["B3", "B2", "B1"], "min": 1000, "max": 3500}

  #now let's select different bands for calculation of indexes 
  nir1 = dataset1.select('B8')
  red1= dataset1.select('B4')
  swir1 = dataset1.select('B11')
  mir1 = dataset1.select('B12')
  nir2= dataset2.select('B8')
  red2= dataset2.select('B4')
  swir2 = dataset2.select('B11')
  mir2 = dataset1.select('B12')
  if (index1.value =='NDVI'):
    ndvi_date1 = nir1.subtract(red1).divide(nir1.add(red1))
    ndvi_date2 = nir2.subtract(red2).divide(nir2.add(red2))
    # ndvi difference 
    ndvi_diff = ndvi_date2.subtract(ndvi_date1)
    # to calculate z -score of the ndvi difference
    print('Computing zscores...')
    ndvi_diff_stats = geemap.image_stats(ndvi_diff, scale=10)
    stats = ndvi_diff_stats.getInfo()
    # we need mean and standard deviation so first calculate the image statstics of ndvi 
    ndvi_mean = stats.get('mean').get('B8')
    ndvi_std = stats.get('std').get('B8')
    zscore = ndvi_diff.subtract(ndvi_mean).divide(ndvi_std)
    # creating masks classification zones for z score and vizualizing it 
    zones = zscore.gt(-2).add(zscore.gt(0)).add(zscore.gt(2))#.add(zscore.gt(-2).And(zscore.lt(2)))
    #define visualization parameters 
    vis_params_zscore = {
    'min': -2,
    'max': 2,
    'palette': ['blue','white','Red','yellow']}

    vis_params = {
    'min': -1,
    'max': 1,
    'palette': ['blue','white','green']}
    palette = ['FF0000', '0000FF', '00FF00','FF0000']
    print('Adding the layers...')
    Map.addLayer(dataset1, vis, 'Sentinel_Date1')
    Map.addLayer(dataset2, vis, 'Sentinel_Date2')
    Map.addLayer(ndvi_diff, vis_params, 'NDVI_difference')
    Map.addLayer(zscore, vis_params_zscore, 'Zscore')
    Map.addLayer(zones, {'min': 0, 'max': 3, 'palette': palette}, 'zones')
    Map.addLayer(pt,{},'Location')
    Map.centerObject(ee.Geometry.Point([longitude, latitude]), 16)
  elif (index1.value =='NDBI'):
    ndbi_date1 = swir1.subtract(nir1).divide(swir1.add(nir1))
    ndbi_date2 = swir2.subtract(red2).divide(swir2.add(nir2))
    ndbi_diff = ndbi_date2.subtract(ndbi_date1)
    ndbi_diff_stats = geemap.image_stats(ndbi_diff, scale=10)
    stats = ndbi_diff_stats.getInfo()
    ndbi_mean = stats.get('mean').get('B11')
    ndbi_std = stats.get('std').get('B11')
    print('Computing zscores...')
    zscore = ndbi_diff.subtract(ndbi_mean).divide(ndbi_std)
    # creating masks classification zones for z score and vizualizing it 
    zones = zscore.gt(-2).add(zscore.gt(0)).add(zscore.gt(2))#.add(zscore.gt(-2).And(zscore.lt(2)))
    vis_params_zscore = {
    'min': -2,
    'max': 2,
    'palette': ['blue','white','Red','yellow']}
    vis_params = {
    'min': -1,
    'max': 1,
    'palette': ['black','white','brown']}
    palette = ['FF0000', '0000FF', '00FF00','FF0000']
    print('Adding the layers...')
    Map.addLayer(dataset1, vis, 'Sentinel_Date1')
    Map.addLayer(dataset2, vis, 'Sentinel_Date2')
    Map.addLayer(ndbi_diff, vis_params, 'NDBI_difference')
    Map.addLayer(zscore, vis_params_zscore, 'Zscore')
    Map.addLayer(zones, {'min': 0, 'max': 3, 'palette': palette}, 'zones')
    Map.addLayer(pt,{},'Location')
    Map.centerObject(ee.Geometry.Point([longitude, latitude]), 16)
  else: 
    ndwi_date1 = nir1.subtract(mir1).divide(nir1.add(mir1))
    ndwi_date2 = nir2.subtract(mir2).divide(nir2.add(mir2))
    # ndvi difference 
    ndwi_diff = ndwi_date2.subtract(ndwi_date1)
    # to calculate z -score of the ndvi difference
    print('Computing zscores...')
    ndwi_diff_stats = geemap.image_stats(ndwi_diff, scale=10)
    stats = ndwi_diff_stats.getInfo()
    # we need mean and standard deviation so first calculate the image statstics of ndvi 
    ndwi_mean = stats.get('mean').get('B8')
    ndwi_std = stats.get('std').get('B8')
    zscore = ndwi_diff.subtract(ndwi_mean).divide(ndwi_std)
    # creating masks classification zones for z score and vizualizing it 
    zones = zscore.gt(-2).add(zscore.gt(0)).add(zscore.gt(2))#.add(zscore.gt(-2).And(zscore.lt(2)))
    #define visualization parameters 
    vis_params_zscore = {
    'min': -2,
    'max': 2,
    'palette': ['blue','white','Red','yellow']}

    vis_params = {
    'min': -1,
    'max': 1,
    'palette': ['green','white','grey']}
    palette = ['FF0000', '0000FF', '00FF00','FF0000']
    print('Adding the layers...')
    Map.addLayer(dataset1, vis, 'Sentinel_Date1')
    Map.addLayer(dataset2, vis, 'Sentinel_Date2')
    Map.addLayer(ndwi_diff, vis_params, 'NDWI_difference')
    Map.addLayer(zscore, vis_params_zscore, 'Zscore')
    Map.addLayer(zones, {'min': 0, 'max': 3, 'palette': palette}, 'zones')
    Map.addLayer(pt,{},'Location')
    Map.centerObject(ee.Geometry.Point([longitude, latitude]), 16)

In [9]:
butt = widgets.Button(description='Apply')
out = widgets.Output()
butt.on_click(on_button_clicked) 

In [ ]:
output_widget = widgets.Output(layout={'border': '2px solid black'})
output_control = WidgetControl(widget=output_widget, position='bottomright')
Map.add_control(output_control)
with output_widget:
  output_widget.clear_output()
  display(start_date)
  display(end_date)
  display(location1)
  display(index1)
  display(butt)
Map